In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Copy dataset.zip file in Google Collab storage
!cp drive/"My Drive"/"Colab Notebooks"/dataset.zip /tmp/dataset.zip

In [ ]:
# Import required modules
import numpy as np
import pandas as pd
import os
import zipfile
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Extract data from dataset.zip file
local_zip = '/tmp/dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')

Using TensorFlow backend.


In [ ]:
# Build CNN model
classifier = Sequential()
# Convolution layer
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
# Pooling layer
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Convolution layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
# Pooling layer
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Flattening layer
classifier.add(Flatten())
# Dense layer
classifier.add(Dense(units = 128, activation = 'relu'))
# Dense layer
classifier.add(Dense(units = 2, activation = 'sigmoid'))
# Using Adam optimizer and Cross Entropy loss function
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.summary()

In [ ]:
# Utility to generate training data
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.20)

# Utility to generate test data
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Generate Training set
train_generator = train_datagen.flow_from_directory('/tmp/dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 40,
                                                 color_mode = 'rgb',
                                                 subset = 'training',
                                                 shuffle = True,
                                                 class_mode = 'categorical',
                                                 seed = 47)

Found 1760 images belonging to 2 classes.


In [ ]:
# Generate Validation set
valid_generator = train_datagen.flow_from_directory('/tmp/dataset/training_set',
                                            target_size = (64, 64),
                                            batch_size = 40,
                                            color_mode = 'rgb',
                                            subset = 'validation',
                                            shuffle = True,
                                            class_mode = 'categorical',
                                            seed = 47)

Found 440 images belonging to 2 classes.


In [ ]:
# Generate Test set
test_generator = test_datagen.flow_from_directory(
    directory='/tmp/dataset/test_set',
    target_size=(64, 64),
    color_mode='rgb',
    batch_size=32,
    class_mode=None,
    shuffle=False,
    seed=47
)

Found 384 images belonging to 1 classes.


In [ ]:
# Training the classifier
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
classifier.fit_generator(train_generator,
                         steps_per_epoch = STEP_SIZE_TRAIN,
                         epochs = 10,
                         validation_data = valid_generator,
                         validation_steps = STEP_SIZE_VALID)

Epoch 1/10
44/44 [==============================] - 13s 294ms/step - loss: 0.6712 - accuracy: 0.5767 - val_loss: 0.5479 - val_accuracy: 0.6489
Epoch 2/10
44/44 [==============================] - 13s 287ms/step - loss: 0.4733 - accuracy: 0.7699 - val_loss: 0.2215 - val_accuracy: 0.8659
Epoch 3/10
44/44 [==============================] - 13s 286ms/step - loss: 0.2572 - accuracy: 0.9045 - val_loss: 0.1874 - val_accuracy: 0.9318
Epoch 4/10
44/44 [==============================] - 13s 288ms/step - loss: 0.1465 - accuracy: 0.9446 - val_loss: 0.1694 - val_accuracy: 0.9466
Epoch 5/10
44/44 [==============================] - 13s 288ms/step - loss: 0.1518 - accuracy: 0.9438 - val_loss: 0.1623 - val_accuracy: 0.9500
Epoch 6/10
44/44 [==============================] - 13s 285ms/step - loss: 0.0981 - accuracy: 0.9656 - val_loss: 0.0318 - val_accuracy: 0.9682
Epoch 7/10
44/44 [==============================] - 13s 285ms/step - loss: 0.0890 - accuracy: 0.9625 - val_loss: 0.1344 - val_accuracy: 0.9545

In [ ]:
# Calculate accuracy and loss on validation set
score = classifier.evaluate_generator(valid_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.08227900415658951
Test accuracy: 0.9681817889213562


In [ ]:
# Predict output for test set
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
predict=classifier.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

12/12 [==============================] - 1s 82ms/step


In [ ]:
# Save predictions to results.csv file
predicted_class_indices=pred.argmax(axis=-1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)

In [ ]:
# Save the model
classifier.save("model.h5")